In [ ]:
from  tiny.util import *
df = get_start_closed()
df = df[:20000]
#df = df[df.device=='f98472c327217ce72f596b0382c62f07']
df = split_days_all(df)
df = extend_time(df, span_no=2)
print(df.shape)


In [ ]:
df.sort_values('start')
df.device.unique()
print(df.shape)

In [ ]:
cut=10000
df1 = extend_package_df(df[:cut])
df2 = extend_package_df(df[cut:])


In [ ]:

df_all = pd.concat([df1,df2])
df_all
print(df_all.shape)
print(len(df_all.columns.unique()))

In [ ]:
df2

In [ ]:
#df2.set_index('device', inplace=True)
df.sort_index(ascending=False)


In [ ]:
split.head()

In [ ]:
print(pd.to_datetime('1488332491588', unit='ms'))  #fc6383d2bdb463fd01db6731594a6d36	07e967d75aab2f6a52c558695a572a7c
print(pd.to_datetime('1488332506604', unit='ms'))
print(pd.to_datetime('1488363700875', unit='ms'))
print(pd.to_datetime('1488363707487', unit='ms'))

In [ ]:
tmp = extend_time(tmp,2)
#tmp['start'].astype
#tmp['start'] = pd.to_datetime(tmp['start'])
#tmp['start'].dt.date

In [ ]:

# def extend_percent(df):
#     total = get_percent_duration(df, ['device'], 'total')

#     max_week = get_max_week(df)

#     merge = df.merge(max_week, on=['device', 'weekbegin'])

#     max = get_percent_duration(merge, ['device'], prefix='max')

#     return pd.concat( [total, max], axis=1 )


# total = get_percent_duration(df, ['device'], 'total')


# df = tmp
# max_week = get_max_week(df)
# max_week.head()
# merge = df.merge(max_week, on=['device', 'weekbegin'])

 
# def get_percent_duration(df, groupby=['device', 'weekday'], prefix=None):
#     prefix = groupby[-1] if prefix is None else prefix
#     columns = [key for key in df.columns if 'span_' in key]
#     gp_map = [(key, 'sum') for key in columns if 'span_' in key]
#     gp_map = dict(gp_map)
#     # print(gp_map)

#     gp_map['package'] = 'nunique'
#     gp_map['start_base'] = 'nunique'
#     df = df.groupby(groupby).agg(gp_map)
#     #     df = df.groupby(groupby).agg({'span_0':'sum','span_1':'sum','span_2':'sum',
#     #                                                  'span_3':'sum','package':'nunique' })
#     df['total'] = df[[key for key in columns if 'span_' in key]].sum(axis=1)

#     for col in columns:
#         df[f'{col}_p'] = round(df[col] / df['total'], 3)

#     df.rename({'package':'package_cnt', 'start_base':'start_base_cnt'}, axis=1, inplace=True)

#     df.columns = [f'{prefix}_{key}' for key in df.columns]
#     return df

per = extend_percent(tmp)
per.head()

# max_week = get_max_week(tmp)
# max_week

# merge= tmp.merge(max_week, on = ['device','weekbegin'])

# print(len(merge))

# get_percent_duration(merge, ['device'], prefix='max')


    

In [ ]:
per
#tmp[tmp.device=='fc6383d2bdb463fd01db6731594a6d36'].sort_values('start')

In [ ]:
print('yes')

In [ ]:
#tmp['start_base']  = tmp['start'].dt.date
# tmp = tmp[tmp.device.isin(['fff885dafce62310951c835247f30a38',
# 'fc65f1731b785a8ce82d20a30dee9259'])]

df = tmp.groupby(['device','weekbegin']).package.nunique().to_frame().reset_index()

# .to_frame().reset_index()
# df.sort_values(['device', 'package'])

df.sort_values(by=['device', 'package', 'weekbegin'], ascending=False).groupby('device').nth(0)


In [ ]:
xx = df



In [ ]:


import numpy as np
from tqdm import tqdm
from sklearn.decomposition import LatentDirichletAllocation

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

path = './input/'
data = pd.DataFrame()
sex_age = pd.read_excel('./input/性别年龄对照表.xlsx')
deviceid_packages = pd.read_csv(path + 'deviceid_packages.tsv', sep='\t', names=['device', 'apps'])
deviceid_test = pd.read_csv(path + 'deviceid_test.tsv', sep='\t', names=['device'])
deviceid_train = pd.read_csv(path + 'deviceid_train.tsv', sep='\t', names=['device', 'sex', 'age'])

deviceid_train = pd.concat([deviceid_train, deviceid_test])

deviceid_packages['apps'] = deviceid_packages['apps'].apply(lambda x: x.split(','))
deviceid_packages['app_lenghth'] = deviceid_packages['apps'].apply(lambda x: len(x))

deviceid_packages.head()
# apps = deviceid_packages['apps'].apply(lambda x: ' '.join(x)).tolist()

# vectorizer = CountVectorizer()
# transformer = TfidfTransformer()
# cntTf = vectorizer.fit_transform(apps)
# tfidf = transformer.fit_transform(cntTf)
# #word = vectorizer.get_feature_names()
# weight = tfidf.toarray()
# df_weight = pd.DataFrame(weight)
# feature = df_weight.columns
# df_weight['sum'] = 0
# for f in tqdm(feature):
#     df_weight['sum'] += df_weight[f]
# deviceid_packages['tfidf_sum'] = df_weight['sum']

# lda = LatentDirichletAllocation(n_topics=5,
#                                 learning_offset=50.,
#                                 random_state=666)
# docres = lda.fit_transform(cntTf)

# deviceid_packages = pd.concat([deviceid_packages, pd.DataFrame(docres)], axis=1)

# temp = deviceid_packages.drop('apps', axis=1)
# deviceid_train = pd.merge(deviceid_train, temp, on='device', how='left')

# deviceid_train['sex'] = deviceid_train['sex'].apply(lambda x: str(x))
# deviceid_train['age'] = deviceid_train['age'].apply(lambda x: str(x))




In [ ]:
from tiny.lda import *
get_lda_from_usage()

The input df#853 before split, with max duration:27.0 and  threshold_days@1
The input df#853 before split, with max duration:26.0 and  threshold_days@1
The input df#852 before split, with max duration:25.0 and  threshold_days@1
The input df#850 before split, with max duration:24.0 and  threshold_days@1
The input df#848 before split, with max duration:23.0 and  threshold_days@1
The input df#847 before split, with max duration:22.0 and  threshold_days@1
The input df#847 before split, with max duration:21.0 and  threshold_days@1
The input df#846 before split, with max duration:20.0 and  threshold_days@1
The input df#846 before split, with max duration:19.0 and  threshold_days@1
The input df#845 before split, with max duration:18.0 and  threshold_days@1
The input df#844 before split, with max duration:17.0 and  threshold_days@1
The input df#842 before split, with max duration:16.0 and  threshold_days@1
The input df#841 before split, with max duration:15.0 and  threshold_days@1
The input df

2018-09-12 22:14:48,462 util_log.py[38] INFO cost:18.64 sec: ==='split_days_all' end ([], {}) 
2018-09-12 22:14:49,436 util_log.py[33] INFO Begin to run get_start_closed with:['./output/start_close/deviceid_package_start_close_40_14_13182781_14124453.csv'], {}


Device_Package: convert (987077, 10) to (1843, 3198) 
Try to summary file:./output/start_close/deviceid_package_start_close_40_14_13182781_14124453.csv
Sort the df#941672 by device(begin)


2018-09-12 22:14:50,966 util_log.py[38] INFO cost: 1.53 sec: ==='get_start_closed' end (['./output/start_close/deviceid_package_start_close_40_14_13182781_14124453.csv'], {}) 
2018-09-12 22:14:50,968 util_log.py[33] INFO Begin to run split_days_all with:[], {}


Sort the df by device(end)
Remove 0 records data
Out loop: The original Df size is 941672
The input df#941672 before split, with max duration:17250.000719328706 and  threshold_days@50
The input df#24 before split, with max duration:17200.528096377315 and  threshold_days@50
The input df#23 before split, with max duration:17150.528096377315 and  threshold_days@50
The input df#23 before split, with max duration:17100.528096377315 and  threshold_days@50
The input df#23 before split, with max duration:17050.528096377315 and  threshold_days@50
The input df#23 before split, with max duration:17000.528096377315 and  threshold_days@50
The input df#23 before split, with max duration:16950.528096377315 and  threshold_days@50
The input df#23 before split, with max duration:16900.528096377315 and  threshold_days@50
The input df#23 before split, with max duration:16850.528096377315 and  threshold_days@50
The input df#23 before split, with max duration:16800.528096377315 and  threshold_days@50
The in

...

max duration:677.2175743634259 with small threshold:50
The output df#42 after split
max duration:727.2175743634259 with small threshold:50
The output df#45 after split
max duration:777.2175743634259 with small threshold:50
The output df#48 after split
max duration:827.2175743634259 with small threshold:50
The output df#51 after split
max duration:877.2175743634259 with small threshold:50
The output df#54 after split
max duration:927.2175743634259 with small threshold:50
The output df#57 after split
max duration:977.2175743634259 with small threshold:50
The output df#60 after split
max duration:1027.217574363426 with small threshold:50
The output df#63 after split
max duration:1077.217574363426 with small threshold:50
The output df#66 after split
max duration:1127.217574363426 with small threshold:50
The output df#69 after split
max duration:1177.217574363426 with small threshold:50
The output df#72 after split
max duration:1227.217574363426 with small threshold:50
The output df#75 afte

max duration:14677.217574363425 with small threshold:50
The output df#888 after split
max duration:14727.217574363425 with small threshold:50
The output df#894 after split
max duration:14777.217574363425 with small threshold:50
The output df#900 after split
max duration:14827.217574363425 with small threshold:50
The output df#906 after split
max duration:14877.217574363425 with small threshold:50
The output df#912 after split
max duration:14927.217574363425 with small threshold:50
The output df#918 after split
max duration:14977.217574363425 with small threshold:50
The output df#924 after split
max duration:15027.217574363425 with small threshold:50
The output df#930 after split
max duration:15077.217574363425 with small threshold:50
The output df#936 after split
max duration:15127.217574363425 with small threshold:50
The output df#942 after split
max duration:15177.217574363425 with small threshold:50
The output df#948 after split
max duration:15227.217574363425 with small threshold:5

The output df#6907 after split
max duration:6.0 with small threshold:1
The output df#8289 after split
max duration:7.0 with small threshold:1
The output df#9671 after split
max duration:8.0 with small threshold:1
The output df#11053 after split
max duration:9.0 with small threshold:1
The output df#12435 after split
max duration:10.0 with small threshold:1
The output df#13817 after split
max duration:11.0 with small threshold:1
The output df#15199 after split
max duration:12.0 with small threshold:1
The output df#16581 after split
max duration:13.0 with small threshold:1
The output df#17964 after split
max duration:14.0 with small threshold:1
The output df#19348 after split
max duration:15.0 with small threshold:1
The output df#20734 after split
max duration:16.0 with small threshold:1
The output df#22121 after split
max duration:17.0 with small threshold:1
The output df#23510 after split
max duration:18.0 with small threshold:1
The output df#24900 after split
max duration:19.0 with sma

2018-09-12 22:20:40,400 util_log.py[38] INFO cost:20.87 sec: ==='split_days_all' end ([], {}) 


Device_Package: convert (1016983, 10) to (1931, 3385) 
Share of device package usage is:(72727, 35000)


In [ ]:
tmp = extend_cols(tmp)
tmp, tmp_new = split_days(tmp)

tmp.head()

In [ ]:
tmp, tmp_new = split_days(tmp)
tmp.head()

In [ ]:
trainn_label = get_train_label()
trainn_label.head()

In [ ]:
from  tiny.util import *

tmp = extend_feature( span_no=2, trunc_long_time=False, mini=True)

In [ ]:
tmp.head()

In [ ]:
tmp = extend_time_span(version=1, trunc_long_time=False, mini=True, groupby=['device'], prefix='tol')
tmp.head()

In [1]:
from tiny.lda import *
from  tiny.util import *

# New add
# deviceid_train.rename({'device_id':'device'}, axis=1, inplace=True)
deviceid_train = get_lda_feature()
deviceid_train = extend_feature(span_no=6, input=deviceid_train, trunc_long_time=False)


2018-09-13 10:44:01,516 util_cache_file.py[68] DEBUG fn:get_lda_feature, para:[], kw:{}
2018-09-13 10:44:01,519 util_cache_file.py[20] DEBUG try to read cache from file:./cache/get_lda_feature_[]_{}.csv, type:csv
/Users/lali2/dev/python/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2862: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
2018-09-13 10:44:01,771 util_cache_file.py[30] DEBUG Return 72727 resut from file cache:./cache/get_lda_feature_[]_{}.csv
2018-09-13 10:44:01,773 util_log.py[33] INFO Begin to run extend_time_span with:[], {'version': 2, 'trunc_long_time': False, 'mini': False, 'groupby': ['device'], 'prefix': 'tol'}
2018-09-13 10:44:01,774 util_cache_file.py[68] DEBUG fn:extend_time_span, para:[], kw:{'version': 2, 'trunc_long_time': False, 'mini': False, 'groupby': ['device'], 'prefix': 'tol'}
2018-09-13 10:44:01,776 util_cache_file.py[20] DE

columns before reduce:Index(['device', 'tol_span24_0_sum', 'tol_span24_0_count', 'tol_span24_1_sum',
       'tol_span24_1_count', 'tol_span24_2_sum', 'tol_span24_2_count',
       'tol_span24_3_sum', 'tol_span24_3_count', 'tol_span24_4_sum',
       'tol_span24_4_count', 'tol_span24_5_sum', 'tol_span24_5_count',
       'tol_span24_6_sum', 'tol_span24_6_count', 'tol_span24_7_sum',
       'tol_span24_7_count', 'tol_span24_8_sum', 'tol_span24_8_count',
       'tol_span24_9_sum', 'tol_span24_9_count', 'tol_span24_10_sum',
       'tol_span24_10_count', 'tol_span24_11_sum', 'tol_span24_11_count',
       'tol_span24_12_sum', 'tol_span24_12_count', 'tol_span24_13_sum',
       'tol_span24_13_count', 'tol_span24_14_sum', 'tol_span24_14_count',
       'tol_span24_15_sum', 'tol_span24_15_count', 'tol_span24_16_sum',
       'tol_span24_16_count', 'tol_span24_17_sum', 'tol_span24_17_count',
       'tol_span24_18_sum', 'tol_span24_18_count', 'tol_span24_19_sum',
       'tol_span24_19_count', 'tol_span2

2018-09-13 10:44:03,521 util_log.py[33] INFO Begin to run extend_brand_pkg with:[], {}
2018-09-13 10:44:03,524 util_log.py[33] INFO Begin to run get_brand with:[], {}
2018-09-13 10:44:03,634 util_log.py[33] INFO Begin to run convert_label_encode with:[['device']], {}
2018-09-13 10:44:03,732 util_log.py[38] INFO cost: 0.10 sec: ==='convert_label_encode' end ([['device']], {}) 
2018-09-13 10:44:03,736 util_log.py[38] INFO cost: 0.21 sec: ==='get_brand' end ([], {}) 
2018-09-13 10:44:03,848 util_log.py[38] INFO cost: 0.33 sec: ==='extend_brand_pkg' end ([], {}) 


column list:Index(['device', 'tol_pkg_cnt_nunique', 'tol_day_cnt_min', 'tol_day_cnt_max',
       'tol_day_cnt_nunique', 'tol_total_sum_', 'tol_total_count_',
       'tol_span_0', 'tol_span_1', 'tol_span_2', 'tol_span_3', 'tol_span_4',
       'tol_span_5', 'tol_total_sum__p', 'tol_total_count__p'],
      dtype='object')


In [3]:
deviceid_train[['tol_day_cnt_min', 'tol_day_cnt_max']].head()

,tol_day_cnt_min,tol_day_cnt_max
0,2017-02-28,2017-03-31
1,2017-02-27,2017-03-31
2,2017-02-28,2017-03-26
3,2017-02-14,2017-03-30
4,2017-02-28,2017-03-31
